In [1]:
#################################### Yelp labeled dataset1 ingestion into staging #######################################

In [2]:
dbutils.fs.mkdirs("/mnt/etl/parquet/labeled/Lreviewdata")
dbutils.fs.mkdirs("/mnt/etl/parquet/unlabeled/ULreviewdata")

Out[ 29 ]: True

In [3]:
#train, test = data.randomSplit([0.9, 0.1], seed=12345)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, size, when, broadcast
from pyspark.sql.functions import lit

y_reviewcontent = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpnyc/20190212/reviewcontent/")
y_reviewmetadata = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpnyc/20190212/reviewmetadata/")
y_userIdMapping = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpnyc/20190212/userIdMapping/")
y_productIdMapping = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpnyc/20190212/productIdMapping/")

temp_df1 = y_reviewcontent.select('Reviewuser_id', 'Reviewprod_id', 'Reviewdate','Text').withColumn("id", F.monotonically_increasing_id())
#temp_df2 = rdd_join.select(col("label"))
temp_df2 = y_reviewmetadata.select('metauser_id', 'metaprod_id','rating','label','metadate').withColumn("id", F.monotonically_increasing_id())
temp_df1.createOrReplaceTempView('temp_df1')
temp_df1=spark.sql('select row_number() over (order by "id") as key, * from temp_df1').drop("id")
temp_df2.createOrReplaceTempView('temp_df2')
temp_df2=spark.sql('select row_number() over (order by "id") as key, * from temp_df2').drop("id")

Combined_dataset = temp_df1.join(broadcast(temp_df2), "key", "inner").withColumn("label", when(temp_df2["label"] == "-1", 0).otherwise(1)).select('Text','rating','label')
Combined_dataset.registerTempTable("DF")
#Combined_dataset.cache()
Combined_dataset.count()
#Join with productidmapping to get productname
temp_df1 = y_reviewcontent.select('Reviewuser_id', 'Reviewprod_id', 'Reviewdate').withColumn("id", F.monotonically_increasing_id())
temp_df2 = Combined_dataset.select('rating','label','Text').withColumn("id", F.monotonically_increasing_id())
temp_df1.createOrReplaceTempView('temp_df1')
temp_df1=spark.sql('select row_number() over (order by "id") as key, * from temp_df1').drop("id")
temp_df2.createOrReplaceTempView('temp_df2')
temp_df2=spark.sql('select row_number() over (order by "id") as key, * from temp_df2').drop("id")
yelp_dataset = temp_df2.join(broadcast(temp_df1), "key", "inner").select('key',temp_df1.Reviewuser_id.alias('user_id'), temp_df1.Reviewprod_id.alias('product_id'), 'Reviewdate','Text','rating','label')

yelp_dataset = yelp_dataset.join(broadcast(y_productIdMapping), "product_id", "left").select('key','user_id','productname', 'Reviewdate','Text','rating','label')
yelp_dataset = yelp_dataset.join(broadcast(y_userIdMapping), "user_id", "left").select('key','reviewername','productname', 'Reviewdate','Text','rating','label')

yelp_dataset = yelp_dataset.withColumn('sourcetype', lit("yelp"))
yelp_dataset = yelp_dataset.withColumn('runtype', lit("Batch"))

In [4]:
yelp_dataset.count()

Out[ 23 ]: 359052

In [5]:
#Ingestion to staging layer
yelp_dataset.write.mode("overwrite").format("parquet").save("/mnt/etl/parquet/labeled/Lreviewdata/")

In [6]:
%fs ls /mnt/etl/parquet/labeled/Lreviewdata

path,name,size
dbfs:/mnt/etl/parquet/labeled/Lreviewdata/_SUCCESS,_SUCCESS,0
dbfs:/mnt/etl/parquet/labeled/Lreviewdata/_committed_4321657782449707820,_committed_4321657782449707820,122
dbfs:/mnt/etl/parquet/labeled/Lreviewdata/_started_4321657782449707820,_started_4321657782449707820,0
dbfs:/mnt/etl/parquet/labeled/Lreviewdata/part-00000-tid-4321657782449707820-11017f7c-8231-41d8-8925-e8a98f09983d-262-c000.snappy.parquet,part-00000-tid-4321657782449707820-11017f7c-8231-41d8-8925-e8a98f09983d-262-c000.snappy.parquet,149960499


In [7]:
############################################### Amazon labeled dataset ingestion into staging ##############################################

In [8]:
amazon_reviewcontent = spark.read.parquet("/mnt/datalake/parquet/batch/unlabeled/amazon/20190212/camera/")

In [9]:
amazon_reviewcontent.show(2)

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
 asin|helpful|overall| reviewText| reviewTime| reviewerID| reviewerName| summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
B00C80S4W6| [2, 2]| 5.0|This is a great q...|03 10, 2014|A18U49406IPPIJ| Deon|Great Way To Prot...| 1394409600|
B00C80S4W6| [2, 2]| 5.0|Very,very well ma...| 07 5, 2013| ADKXBC6D7XOYE|Desdemona Sesay|BEST SCREEN PROTE...| 1372982400|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
only showing top 2 rows

In [10]:
UL_amazon_dataset = amazon_reviewcontent.withColumn("id", F.monotonically_increasing_id())\
                    .withColumn('sourcetype', lit('amazon')).withColumn('runtype',lit('batch'))\
                    .withColumn('productname',lit('Cellphone'))\
                    .select(amazon_reviewcontent.reviewerName.alias('reviewername'),\
                      'productname',amazon_reviewcontent.reviewTime.alias('Reviewdate'),\
                       amazon_reviewcontent.reviewText.alias('Text'),\
                       amazon_reviewcontent.overall.alias('rating'),\
                       'sourcetype','runtype')

In [11]:
UL_amazon_dataset.show(3)

+---------------+-----------+-----------+--------------------+------+----------+-------+
 reviewername|productname| Reviewdate| Text|rating|sourcetype|runtype|
+---------------+-----------+-----------+--------------------+------+----------+-------+
 Deon| Cellphone|03 10, 2014|This is a great q...| 5.0| amazon| batch|
Desdemona Sesay| Cellphone| 07 5, 2013|Very,very well ma...| 5.0| amazon| batch|
 Justin Shorey| Cellphone|05 10, 2014|These are the abs...| 1.0| amazon| batch|
+---------------+-----------+-----------+--------------------+------+----------+-------+
only showing top 3 rows

In [12]:
UL_amazon_dataset.createOrReplaceTempView('temp_df2')
UL_amazon_dataset = spark.sql('select row_number() over (order by "id")  as key, * from temp_df2').select('key','reviewername','productname','Reviewdate','Text','rating','sourcetype','runtype')

In [13]:
#Ingestion to staging layer
UL_amazon_dataset.write.mode("overwrite").format("parquet").save("/mnt/etl/parquet/unlabeled/ULreviewdata")

In [14]:
yelp_dataset.show(5)

+---+--------------------+-----------+----------+----+------+-----+----------+-------+
key| reviewername|productname|Reviewdate|Text|rating|label|sourcetype|runtype|
+---+--------------------+-----------+----------+----+------+-----+----------+-------+
 1|qmUCoGss3hdNmye2h...| Sottocasa| 5.0| 1| 5.0| 1| yelp| Batch|
 2|og3ItQKKLqUGGhXds...| Sottocasa| 5.0| 1| 5.0| 1| yelp| Batch|
 3|7N1gmp5e80SH86Ak-...| Sottocasa| 5.0| 1| 5.0| 1| yelp| Batch|
 4|2gmpQfplDLtRJG2PV...| Sottocasa| 5.0| 1| 5.0| 1| yelp| Batch|
 5|aOZBgrf9KJyhvRqLO...| Sottocasa| 5.0| 1| 5.0| 1| yelp| Batch|
+---+--------------------+-----------+----------+----+------+-----+----------+-------+
only showing top 5 rows

In [15]:
amazon_dataset = spark.read.parquet("/mnt/etl/parquet/unlabeled/ULreviewdata")

In [16]:
amazon_dataset.show(5)

+---+---------------+-----------+-----------+--------------------+------+----------+-------+
key| reviewername|productname| Reviewdate| Text|rating|sourcetype|runtype|
+---+---------------+-----------+-----------+--------------------+------+----------+-------+
 1| Deon| Cellphone|03 10, 2014|This is a great q...| 5.0| amazon| batch|
 2|Desdemona Sesay| Cellphone| 07 5, 2013|Very,very well ma...| 5.0| amazon| batch|
 3| Justin Shorey| Cellphone|05 10, 2014|These are the abs...| 1.0| amazon| batch|
 4| MoHaMeD| Cellphone|12 16, 2013|Best choiceGood q...| 5.0| amazon| batch|
 5| Plc| Cellphone| 10 7, 2013|They go on very e...| 2.0| amazon| batch|
+---+---------------+-----------+-----------+--------------------+------+----------+-------+
only showing top 5 rows

In [17]:
##################################### Yelp labeled dataset ingestion into staging ########################################

In [18]:
Lreviewdata = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata")
Lreviewdata.count()

Out[ 44 ]: 359052

In [19]:
y_hotreview = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpchi/20190212/review_yelpHotelData")
y_metahotreview = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpchi/20190212/meta_yelpHotelData")
y_resreview = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpnyc/20190212/userIdMapping/")
y_metaresreview = spark.read.parquet("/mnt/datalake/parquet/batch/labeled/yelp/yelpchi/20190212/review_yelpResData")

In [20]:
temp_df1 = y_hotreview.select('Text').withColumn("id", F.monotonically_increasing_id())
#temp_df2 = rdd_join.select(col("label"))
temp_df2 = y_metahotreview.select(y_metahotreview.date.alias('Reviewdate'), y_metahotreview.reviewer_id.alias('reviewername'),y_metahotreview.reviewer_id.alias('productname'),'label','rating').withColumn("id", F.monotonically_increasing_id())
temp_df1.createOrReplaceTempView('temp_df1')

Lreviewdata = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata")
lastcount = Lreviewdata.count()
temp_df1=spark.sql("select row_number() over (order by id) + {} as key, * from temp_df1".format(lastcount)).drop("id")
temp_df2.createOrReplaceTempView('temp_df2')
temp_df2=spark.sql("select row_number() over (order by id) + {} as key, * from temp_df2".format(lastcount)).drop("id")

yelphotel_dataset = temp_df1.join(broadcast(temp_df2), "key", "inner").withColumn("label", when(temp_df2["label"] == "N", 0).otherwise(1))\
                    .withColumn('sourcetype', lit('yelp'))\
                    .withColumn('runtype',lit('batch'))\
                    .select('key','reviewername','productname','Reviewdate','Text','rating','label','sourcetype','runtype')

In [21]:
yelphotel_dataset.count()

Out[ 47 ]: 5854

In [22]:
#Ingestion to staging layer
yelphotel_dataset.write.mode("append").format("parquet").save("/mnt/etl/parquet/labeled/Lreviewdata/")

In [23]:
Lreviewdata = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata")
Lreviewdata.count()

Out[ 49 ]: 364906

In [24]:
Lreviewdata.createOrReplaceTempView("table_df")
query_latest_rec = """SELECT * FROM table_df ORDER BY key DESC limit 1"""
latest_rec = spark.sql(query_latest_rec)
latest_rec.show()

+------+--------------------+--------------------+----------+--------------------+------+-----+----------+-------+
 key| reviewername| productname|Reviewdate| Text|rating|label|sourcetype|runtype|
+------+--------------------+--------------------+----------+--------------------+------+-----+----------+-------+
364906|xbkzvpD7JF1O2Roza...|xbkzvpD7JF1O2Roza...| 1/8/2010|I stayed here for...| 2| 0| yelp| batch|
+------+--------------------+--------------------+----------+--------------------+------+-----+----------+-------+